In [81]:
import numpy as np
import pandas as pd


df_main = pd.read_csv('main.csv')
df_net_property = pd.read_csv('net_property.csv')
df_seats_property = pd.read_csv('seats_property.csv')
df_type_property = pd.read_csv('type_property.csv')
# df_main = pd.read_csv(r'/content/sample_data/main.csv')
# df_net_property = pd.read_csv('/content/sample_data/net_property.csv')
# df_seats_property = pd.read_csv('/content/sample_data/seats_property.csv')
# df_type_property = pd.read_csv('/content/sample_data/type_property.csv')

## Привожу в порядок посадочные места. Убираю полные дубликаты и убираю дубликаты с минимальной ver_ID

In [83]:
df_seats_property = df_seats_property.sort_values(by=['ID', 'ver_ID'], ascending=[True, False]) \
    .drop_duplicates('ID', keep='first')

# Разбиваю на группы
t_seats = df_seats_property
t_seats['SeatsCount_group'] = pd.cut(t_seats.SeatsCount,
                                     bins=[0, 5, 15, 25, np.inf],
                                     include_lowest=True,
                                     labels=list('1234'))
t_seats

,ID,SeatsCount,ver_ID,SeatsCount_group
673,5882,75,2,4
3965,19627,20,2,3
10104,19657,85,2,4
5032,19678,46,1,4
1449,19681,24,1,3
...,...,...,...,...
2212,375905,34,1,4
8218,375933,5,2,1
8637,375940,10,2,2
2158,375942,0,1,1


Собираю датафрейм с "Сетевые объекты" со специализацией "кафе" и актуальной версией специализации is_actual == True

In [84]:
df_net_property.drop_duplicates(inplace=True) #Убрал одну дублирующуюся запись

In [85]:
df1 = df_main.merge(df_net_property, on='ID', how='left').merge(df_type_property, on='ID', how='left')
df1 = df1.merge(t_seats, on='ID', how='left')

In [86]:
df1 = df1.query('is_actual == True and TypeObject == "кафе" and IsNetObject == "да"')

In [87]:
df1.SeatsCount_group.value_counts()

4    46
2    12
3    10
1     2
Name: SeatsCount_group, dtype: int64

(70, 9)

In [97]:
df_address = pd.read_excel('Список_адресов.xlsx', sheet_name=1, skiprows=[0])
df_ID = pd.read_excel('Список_ID.xlsx', sheet_name=0, skiprows=[0])
df_address.rename(columns={"Unnamed: 0": 'f_address'}, inplace=True)

In [99]:
df_type_property = df_type_property[df_type_property.is_actual == True]

In [100]:
t1 = df_main.merge(df_ID)
t2 = df_main.merge(df_address, left_on='Address', right_on='f_address').drop('f_address', axis=1)
df_f = pd.concat([t1, t2]).drop_duplicates()

In [102]:
df_seats_property = df_seats_property.sort_values(by=['ID', 'ver_ID'], ascending=[True, False]) \
    .drop_duplicates('ID', keep='first')
df_f = df_f.merge(df_net_property, on='ID', how='left').merge(df_seats_property, on='ID', how='left').merge(df_type_property, on='ID', how='left')

In [103]:
df_result = df_f.fillna({'IsNetObject': '', 'SeatsCount': 0, 'TypeObject': 'отсутствует'})[['ID', 'Name', 'Address', 'IsNetObject', 'TypeObject', 'SeatsCount']]
df_result = df_result.sort_values('ID', ascending=False)

In [104]:
translated_col = 'Ид,Наименование,Адрес,Сетевой?,Специализация,КоличМест'.split(',')
df_result.rename(columns={df_result.columns[i]: translated_col[i] for i in range(len(translated_col))}, inplace=True)
df_result = df_result.drop_duplicates()

In [105]:
df_result.to_csv('Ex_6.csv', index=False)